# Est ce que le PSG va gagner la final de la Ligue des champion ?

### Analyse et comparaison de la premiere et deuxieme partie de saions 24/25 du PSG (21/05/25).

### (pytorch)

In [54]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import pandas as pd
from io import StringIO

### Cherchons les datas sur Internet (Web scrabbing). Nous voulons trouver les datas des matchs realiser par le PSG cette saison.

In [55]:
# Options pour rendre le navigateur "invisible"
options = Options()
options.add_argument("--headless")  # mode sans interface
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Lancer Chrome avec le bon driver
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver = webdriver.Edge(service=Service(EdgeChromiumDriverManager().install()))

# Aller sur la page
# url = "https://fbref.com/en/squads/e2d8892c/Paris-Saint-Germain-Stats"  # all tab

# only Ligue 1
# url = "https://fbref.com/en/squads/e2d8892c/2024-2025/matchlogs/c13/schedule/Paris-Saint-Germain-Scores-and-Fixtures-Ligue-1"  

# all competition tab
url = "https://fbref.com/en/squads/e2d8892c/2024-2025/matchlogs/all_comps/schedule/Paris-Saint-Germain-Scores-and-Fixtures-All-Competitions " 
driver.get(url)

# Attendre que la page charge (tu peux ajuster selon le site)
time.sleep(5)

# Récupérer le contenu HTML
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

# Fermer le navigateur
driver.quit()

# Ensuite tu peux parser avec BeautifulSoup comme d'habitude
# print(soup.prettify())


#### Sur cette page il y a enormenent de tableau, nous cherchons celui en particulier qui reprend la saison 24/25 du PSG

In [56]:
tables = soup.find_all("table")
print(f"Nombre de tableaux trouvés : {len(tables)}")

for i, table in enumerate(tables):
    caption = table.find("caption")
    title = caption.text if caption else f"Tableau {i+1} sans caption"
    print(title)
    print("-----")

Nombre de tableaux trouvés : 10
Tableau 1 sans caption
-----
Tableau 2 sans caption
-----
Tableau 3 sans caption
-----
Tableau 4 sans caption
-----
Tableau 5 sans caption
-----
Tableau 6 sans caption
-----
Tableau 7 sans caption
-----
Tableau 8 sans caption
-----
Tableau 9 sans caption
-----
Scores & Fixtures 2024-2025 Paris Saint-Germain: All Competitions Table
-----


#### Apres avoir trouver le tableau souhaite nous le filtrons pour garder les parametres suivants:  Date | Composition | Round | Venue | Result | GF | GA | Opponent | xG | xGA | Poss

In [57]:
# tb_saison = tables[0]  # numéro commence à 0
tb_saison = tables[9]  # numéro commence à 0

caption = table.find("caption")
# print(caption.text) 

# Convertir le tableau en DataFrame en évitant le warning
# saison_df = pd.read_html(StringIO(str(tables[0])))[0]
saison_df = pd.read_html(StringIO(str(tables[9])))[0]
# saison_df.dropna(inplace=True)

saison_filtrer = saison_df.drop(columns = ["Time","Day","Captain","Formation","Referee","Opp Formation","Attendance","Match Report","Notes"])

saison_filtrer.loc[23, "GF"] = 1 # penalty score removed
saison_filtrer.loc[23, "GA"] = 1 # penalty score removed
saison_filtrer.loc[43, "GF"] = 1 # penalty score removed
saison_filtrer.loc[43, "GA"] = 0 # penalty score removed
saison_filtrer.drop(20, inplace=True)
saison_filtrer.drop(41, inplace=True)

# saison_filtrer

colonnes = ["GF", "GA", "xG","xGA","Poss"]  # remplace par tes colonnes
saison_filtrer[colonnes] = saison_filtrer[colonnes].astype(float)

print(saison_filtrer.head())



         Date          Comp         Round Venue Result   GF   GA     Opponent  \
0  2024-08-16       Ligue 1   Matchweek 1  Away      W  4.0  1.0     Le Havre   
1  2024-08-23       Ligue 1   Matchweek 2  Home      W  6.0  0.0  Montpellier   
2  2024-09-01       Ligue 1   Matchweek 3  Away      W  3.0  1.0        Lille   
3  2024-09-14       Ligue 1   Matchweek 4  Home      W  3.0  1.0        Brest   
4  2024-09-18  Champions Lg  League phase  Home      W  1.0  0.0    es Girona   

    xG  xGA  Poss  
0  2.9  0.2  76.0  
1  3.8  0.6  68.0  
2  2.6  1.4  58.0  
3  2.8  1.0  67.0  
4  2.1  0.2  64.0  


#### Nous creer on df pour la premiere et deuxieme partie de saison (separer par la traive hivernale)

In [58]:
#Premiere partie de saison
saison_filtrer["Date"] = pd.to_datetime(saison_filtrer["Date"], format="%Y-%m-%d", errors='coerce')
date_limite = pd.to_datetime("2025-01-01")
first_prt_saison = saison_filtrer[saison_filtrer["Date"] < date_limite].copy()
first_prt_saison.dropna(inplace=True)
# first_prt_saison

#Premiere partie de saison
saison_filtrer["Date"] = pd.to_datetime(saison_filtrer["Date"], format="%Y-%m-%d", errors='coerce')
date_limite = pd.to_datetime("2025-01-01")
second_prt_saison = saison_filtrer[saison_filtrer["Date"] > date_limite].copy()
second_prt_saison.dropna(inplace=True)
second_prt_saison

,Date,Comp,Round,Venue,Result,GF,GA,Opponent,xG,xGA,Poss
25,2025-01-12,Ligue 1,Matchweek 17,Home,W,2.0,1.0,Saint-Étienne,4.1,0.4,59.0
27,2025-01-18,Ligue 1,Matchweek 18,Away,W,2.0,1.0,Lens,1.2,0.7,66.0
28,2025-01-22,Champions Lg,League phase,Home,W,4.0,2.0,eng Manchester City,2.8,1.7,63.0
29,2025-01-25,Ligue 1,Matchweek 19,Home,D,1.0,1.0,Reims,1.2,0.9,78.0
30,2025-01-29,Champions Lg,League phase,Away,W,4.0,1.0,de Stuttgart,2.4,0.8,58.0
31,2025-02-01,Ligue 1,Matchweek 20,Away,W,5.0,2.0,Brest,3.6,0.9,65.0
33,2025-02-07,Ligue 1,Matchweek 21,Home,W,4.0,1.0,Monaco,3.1,0.9,63.0
34,2025-02-11,Champions Lg,Knockout phase play-offs,Away,W,3.0,0.0,fr Brest,2.6,0.9,62.0
35,2025-02-15,Ligue 1,Matchweek 22,Away,W,1.0,0.0,Toulouse,2.4,0.9,76.0
36,2025-02-19,Champions Lg,Knockout phase play-offs,Home,W,7.0,0.0,fr Brest,3.2,0.9,68.0


### On realise une premiere analyse pour resumer braivement chaque partie de saison du PSG.

In [59]:
total_matches = first_prt_saison.shape[0]
wins1, draw1, loss1 = first_prt_saison["Result"].value_counts()
ration1 = round((wins1/total_matches)*100,2)
print(f"Le PSG fait {wins1} victoires sur {total_matches} matchs avec {draw1} match nuls et {loss1} defaites sur la premier partie de saison.")

total_matches = second_prt_saison.shape[0]
wins2, draw2, loss2 = second_prt_saison["Result"].value_counts()
ration2 = round((wins2/total_matches)*100,2)
print(f"Le PSG fait {wins2} victoires sur {total_matches} matchs avec {draw2} match nuls et {loss2} defaites sur la seconde partie de saison.")
print("--------------------------------------------------------------------------------------------------------------------------\n")

 
print(f"Le PSG realise une meilleur deuxieme partie de saison en aillant gagne {ration2}%'de ces matches \ncontre {ration1}%' en premiere partie malgres le fait que l'equipe ai joue plus de matchs. ")

Le PSG fait 14 victoires sur 22 matchs avec 5 match nuls et 3 defaites sur la premier partie de saison.
Le PSG fait 22 victoires sur 28 matchs avec 4 match nuls et 2 defaites sur la seconde partie de saison.
--------------------------------------------------------------------------------------------------------------------------

Le PSG realise une meilleur deuxieme partie de saison en aillant gagne 78.57%'de ces matches 
contre 63.64%' en premiere partie malgres le fait que l'equipe ai joue plus de matchs. 


In [60]:
def moyenne(df,prt_saison):
    moy_b = round(df["GF"].mean(),2)
    moy_c = round(df["GA"].mean(),2)
    moy_xg = round(df["xG"].mean(),2)
    moy_xga = round(df["xGA"].mean(),2)
    moy_poss = round(df["Poss"].mean(),2)
    nb_g = df["GF"].sum()
    nb_a = df["GA"].sum()
    if prt_saison == 1 :
        print("Pour la premiere partie de saion:")
    elif prt_saison == 2 :
        print("Pour la deuxieme partie de saion:")
    print(f"Le PSG a une moyenne de {moy_b} Buts marque ({nb_g}) et {moy_c} But concede ({nb_a}) pour une possession generale de {moy_poss}%  | ({moy_xg}: xG,{moy_xga}: xGA.")


moyenne(first_prt_saison,1)
print("--------------------------------------------------------------------------------------------------------------------------\n")
moyenne(second_prt_saison,2)

Pour la premiere partie de saion:
Le PSG a une moyenne de 2.27 Buts marque (50.0) et 0.91 But concede (20.0) pour une possession generale de 67.14%  | (2.4: xG,0.91: xGA.
--------------------------------------------------------------------------------------------------------------------------

Pour la deuxieme partie de saion:
Le PSG a une moyenne de 2.68 Buts marque (75.0) et 1.07 But concede (30.0) pour une possession generale de 65.07%  | (2.45: xG,1.04: xGA.


# Prediction sur le resultat d'un match.
### Est ce que le PSG va gagner sa finale de ligue de chanpions avec les qualitees que Luis Enrique a donner a sont equipe ?

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch import nn, optim
from torch.utils.data import TensorDataset, DataLoader

In [1]:
# Supposons que ta DataFrame s'appelle `saison_filtrer`
result_map = {"W": 2, "D": 1, "L": 0}
venue_map = {"Away": 0, "Home": 1}

pred_resultat = saison_filtrer.copy()
pred_resultat = pred_resultat.drop(columns = ["GF", "GA"])
pred_resultat["Cible"] = pred_resultat["Result"].map(result_map)
pred_resultat["Venue"] = pred_resultat["Venue"].map(venue_map)

# On garde uniquement les colonnes nécessaires
# features = ["Venue", "GF", "GA", "xG", "xGA", "Poss"]
features = ["Venue", "xG", "xGA", "Poss"]
target = "Cible"

pred_resultat.dropna(subset=features + [target], inplace=True)

X = pred_resultat[features].values.astype("float32")
y = pred_resultat[target].values.astype("int64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train_tensor = torch.tensor(X_train)
y_train_tensor = torch.tensor(y_train)
X_test_tensor = torch.tensor(X_test)
y_test_tensor = torch.tensor(y_test)

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)


class SimpleClassifier(nn.Module):
    def __init__(self, input_size, hidden_size=16, output_size=3):
        super().__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size)
        )

    def forward(self, x):
        return self.model(x)

# model = SimpleClassifier(input_size=6)  # 6 features 
model = SimpleClassifier(input_size=4)  # car tu as 4 colonnes en entrée

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)



for epoch in range(20):
    model.train()
    total_loss = 0
    for batch_X, batch_y in train_loader:
        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}/20, Loss: {total_loss:.4f}")


model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predicted = torch.argmax(outputs, dim=1)
    accuracy = (predicted == y_test_tensor).float().mean().item()
    print(f"Accuracy: {accuracy * 100:.2f}%")




NameError: name 'saison_filtrer' is not defined

#### Teston notre model de predictions, en entrant des valeur dans les parametre suivant nous pourrions determiner si le PSG gagnerai sont match.

In [ ]:
# print("Exemple : match à domicile, prévision de 1 buts marqués, 4 encaissé, 2.4 xG, 2.9 xGA, 50% possession")
print("Exemple : match à domicile, prévision 2.4 xG, 2.9 xGA, 50% possession")

venue = 0.0       # Home
gf = 1.0          # prévision de buts marqués
ga = 4.0          # prévision de buts encaissés
xg = 0.4          # Xg
xga = 6.9         # Xga 
poss = 30.0       # Possession

# new_match = np.array([[venue, gf, ga, xg, xga, poss]], dtype=np.float32)
new_match = np.array([[venue, xg, xga, poss]], dtype=np.float32)

new_match_tensor = torch.tensor(new_match)

with torch.no_grad():
    output = model(new_match_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

label_map = {0: "Défaite", 1: "Nul", 2: "Victoire"}
print(f"Résultat prédit : {label_map[predicted_class]}")


## Aujourd'hui nous savons que le PSG est champion d Europe !!!!!

#### Maintenant que le match est fini et que nousa vons les parametres (match à domicile, xG, xGA,possession), essayons notre model et voyons voir ce qu il donne comme resultat pour la finale.

In [2]:
venue = 1.0       # away
xg = 3.1          # Xg
xga = 0.5         # Xga 
poss = 59.30       # Possession

new_match = np.array([[venue, xg, xga, poss]], dtype=np.float32)

new_match_tensor = torch.tensor(new_match)

with torch.no_grad():
    output = model(new_match_tensor)
    predicted_class = torch.argmax(output, dim=1).item()

label_map = {0: "Défaite", 1: "Nul", 2: "Victoire"}
print(f"Résultat prédit : {label_map[predicted_class]}")

NameError: name 'np' is not defined